In [1]:
import shap
import os
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# import matplotlib.pyplot as plt
import warnings
# import sys
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.model_selection import train_test_split
# from genericpath import isfile
from TrafficLabelling.utils import *


dataset = 'TrafficLabelling/' # TrafficLabelling lycos-ids2017
modo = 'pca'
segunda = 'knn' # mlp
f = 'CPdId' #['CPdId','CPeIe','CPd_dest','C']:
if dataset == 'lycos-ids2017/':
    dataset_clean = pd.read_csv(dataset + 'dataset_clean_exp_atrib.csv', index_col=[0])
    dataset_clean = pd.get_dummies(dataset_clean, columns=['ip_prot'],drop_first=True)

    # Comentar caso use Pd = 6
    dataset_clean_cat = pd.read_csv(dataset + 'pd4.csv', index_col=[0])
    dataset_clean = pd.concat([dataset_clean_cat, dataset_clean.loc[:,np.invert(dataset_clean.columns.str.contains('[sd]port'))]], axis=1)
    del dataset_clean_cat

    Id = dataset_clean.columns.str.contains('[sd]addr')
    Id = dataset_clean.columns[Id].to_list()
    Pd = dataset_clean.columns.str.contains('[sd]port')
    Pd = dataset_clean.columns[Pd].to_list()

    Ie = dataset_clean.columns.str.match('(src_addr)|(dst_addr)')
    Ie = dataset_clean.columns[Ie].to_list()
    Pe = dataset_clean.columns.str.match('(src_port)|(dst_port)')
    Pe = dataset_clean.columns[Pe].to_list()

    C = np.setdiff1d(dataset_clean.columns,Id+Pd+Ie+Pe+['label','flow_id','timestamp']).tolist()
    Cs = ['flow_duration', 'down_up_ratio', 'pkt_len_max', 'bytes_per_s', 'pkt_per_s', 'fwd_pkt_cnt', 'fwd_pkt_len_tot', 'fwd_pkt_len_max'] + dataset_clean.columns[dataset_clean.columns.str.contains('prot')].to_list()

    # comb_f = {'CPe':C+Pe,'CPd'+str(int(len(Pd)/2)):C+Pd,'PeIe':Pe+Ie,'Pd'+str(int(len(Pd)/2))+'Ie':Pd+Ie,'Pd'+str(int(len(Pd)/2))+'Id'+str(int(len(Id)/2)):Pd+Id,'CPd'+str(int(len(Pd)/2))+'Ie':C+Pd+Ie,'CPd'+str(int(len(Pd)/2))+'Id'+str(int(len(Id)/2)):C+Pd+Id,'C':C,'CPeIe':C+Pe+Ie,'CPd'+str(int(len(Pd)/2))+'_dest':C+Pd[-int(len(Pd)/2):],'CPe_dest':C+Pe[-6:],'Cs':Cs,'CsPd'+str(int(len(Pd)/2))+'_dest':Cs+Pd[-int(len(Pd)/2):]}
    comb_f = {'CPdId':C+Pd+Id,'C':C,'CPeIe':C+Pe+Ie,'CPd_dest':C+Pd[-int(len(Pd)/2):],'Cs':Cs,'CsPd'+str(int(len(Pd)/2))+'_dest':Cs+Pd[-int(len(Pd)/2):]} # para manter compatibilidade com o TrafficLabelling
    # f = 'CPd'+str(int(len(Pd)/2))+'Id'+str(int(len(Id)/2))

    rest_cols = comb_f[f]

elif dataset == 'TrafficLabelling/':
    # dataset original

    C = ['Protocol', 'FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets', 'TotalLengthofFwdPackets', 'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin', 'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax', 'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd', 'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd', 'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd', 'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd', 'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'FwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s', 'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength', 'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance', 'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount', 'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount', 'Down/UpRatio', 'AveragePacketSize', 'AvgFwdSegmentSize', 'AvgBwdSegmentSize', 'SubflowFwdPackets', 'SubflowFwdBytes', 'SubflowBwdPackets', 'SubflowBwdBytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'ActiveMean', 'ActiveStd', 'ActiveMax', 'ActiveMin', 'IdleMean', 'IdleStd', 'IdleMax', 'IdleMin']

    Cs = ['Protocol', 'ACKFlagCount', 'ActiveMean', 'ActiveMin', 'AveragePacketSize', 'BwdIATMean', 'BwdPacketLengthMin', 'BwdPacketLengthStd',
                 'BwdPackets/s', 'FwdIATMean', 'FwdIATMin', 'FwdPacketLengthMean', 'FwdPackets/s', 'FwdPSHFlags', 'FlowDuration', 'FlowIATMean',
                 'FlowIATMin', 'FlowIATStd', 'Init_Win_bytes_backward', 'Init_Win_bytes_forward', 'PSHFlagCount', 'SubflowFwdBytes', 'SYNFlagCount',
                 'TotalLengthofFwdPackets']

    Pd = ['sPort0', 'sPort1', 'sPort2', 'sPort3', 'sPort4', 'sPort5', 'dPort0',
           'dPort1', 'dPort2', 'dPort3', 'dPort4', 'dPort5']

    Id = ['sIP0', 'sIP1', 'sIP2', 'sIP3', 'dIP0', 'dIP1', 'dIP2', 'dIP3']

    # Ie = dataset_clean.columns.str.match('(SourceIP)|(DestinationIP)')
    # Ie = dataset_clean.columns[Ie].to_list()
    Ie = ['SourceIP_DNS_int', 'SourceIP_Reverse_proxy', 'SourceIP_Web_Server_int', 'SourceIP_ext', 'DestinationIP_DNS_int', 'DestinationIP_Reverse_proxy', 'DestinationIP_Web_Server_int', 'DestinationIP_b_m_cast', 'DestinationIP_ext']
    # Pe = dataset_clean.columns.str.match('(SourcePort)|(DestinationPort)')
    # Pe = dataset_clean.columns[Pe].to_list()
    Pe = ['SourcePort_444', 'SourcePort_80', 'SourcePort_8080', 'SourcePort_dynamic', 'SourcePort_registered', 'SourcePort_system', 'DestinationPort_444', 'DestinationPort_80', 'DestinationPort_8080', 'DestinationPort_dynamic', 'DestinationPort_registered', 'DestinationPort_system']
    comb_f = {'CsPe':Cs+Pe,'CPe':C+Pe,'CsPd':Cs+Pd,'CPd':C+Pd,'PeIe':Pe+Ie,'PdIe':Pd+Ie,'PdId':Pd+Id,'CsPdIe':Cs+Pd+Ie,'CsPdId':Cs+Pd+Id,'CPdIe':C+Pd+Ie,'CPdId':C+Pd+Id,'C':C,'Cs':Cs,'CPeIe':C+Pe+Ie,'CPd_dest':C+Pd[-6:]}
    # f = sys.argv[2]
    rest_cols = comb_f[f]

    # dataset_clean = pd.read_csv(path + 'dataset_clean_exp_atrib.csv', index_col=[0])
    dataset_clean = pd.read_csv(dataset + 'dataset_clean_exp_atrib.csv', usecols=rest_cols+['label'])
    # dataset_clean = load_chunk('dataset_clean_exp_atrib.csv',s=20000)
    # dataset_clean = dataset_clean[rest_cols+['Label']]
    dataset_clean.rename(columns={'Label':'label'}, inplace=True)

labels = dataset_clean['label']
features = dataset_clean.loc[:, rest_cols].astype('float64')
del dataset_clean
LE = LabelEncoder()
LE.fit(labels)
labels = LE.transform(labels)

features_train_val, features_test, labels_train_val, labels_test = train_test_split(features, labels, test_size=.2, random_state=42, stratify=labels)

features_train, features_val, labels_train, labels_val = train_test_split(features_train_val, labels_train_val, test_size=.2, random_state=42, stratify=labels_train_val)

m_train = features_train.shape[0]
m_val = features_val.shape[0]
m_test = features_test.shape[0]
m = labels.shape[0]

scaler = StandardScaler()
scaler.fit(features_train)

features_train = pd.DataFrame(scaler.transform(features_train), index=features_train.index,columns=features_train.columns)
# features_train_val = scaler.transform(features_train_val)
features_val = pd.DataFrame(scaler.transform(features_val), index=features_val.index,columns=features_val.columns)
features_test = pd.DataFrame(scaler.transform(features_test), index=features_test.index,columns=features_test.columns)

del features_train_val
del labels_train_val

In [2]:
model_path = dataset+'saved_models/bin_'+f
model = tf.keras.models.load_model(model_path+'.h5',compile=False)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

shap_type = 'nn'


shap_path = 'C:/Users/SE8ALUNO/maj_ricardo/google/doutorado/duvidas/'+dataset+'shap/'+f+'/'+shap_type+'/'
# shap_path = 'J:/Meu Drive/SE8ALUNO/dataset/'+dataset+'shap/'+f+'/'+shap_type+'/'

features = {'train':features_train,'val':features_val, 'test':features_test}
labels = {'train':labels_train, 'val':labels_val, 'test':labels_test}



seed = 42


for cj in ['train','val','test']:

#     prediction = model.predict(features[cj]).squeeze() # com gpu
    prediction = tf.squeeze(model(features[cj].values)).numpy()

    msc_p = prediction > 0.5
    msc_fp = msc_p & (labels[cj] == 0)
    if cj == 'train':
        if dataset+f == 'TrafficLabelling/CPdId':
            conf_limiar = prediction[msc_fp].max()
        else:
            conf_limiar = np.percentile(prediction[msc_fp],99, interpolation='higher')
        msc_p_train = msc_p & (prediction <= conf_limiar)
        dl = {}

    msc_p = msc_p & (prediction <= conf_limiar) # diferente do adversarial, pois aqui é só o que está abaixo do limiar
    filename = shap_path+'shap_values_'+cj+'.npy'

    if os.path.exists(filename):
        shap_values = np.load(filename)
    elif shap_type.startswith('kernel'):
        link='identity'
        if '_k' in shap_type:
            background = shap.kmeans(features['train'][msc_p_train],50)
        else:
            background = shap.sample(features['train'][msc_p_train], 100,random_state=seed)
        explainer = shap.KernelExplainer(model.predict, background, link=link)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            shap_values = explainer.shap_values(features[cj][msc_p])[0]
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        np.save(filename, shap_values)
        mail_content = '{0}, {1}, {2} ok!!'.format(shap_type,f,cj)
        mensagem(mail_content)
    elif shap_type.startswith('nn'):
        # explainer = shap.DeepExplainer(model.predict, features['train'][msc_p_train].values)
        explainer = shap.DeepExplainer(model, shap.sample(features['train'][msc_p_train], 2500,random_state=seed).values)
        shap_values = explainer.shap_values(features[cj][msc_p].values)[0]
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        np.save(filename, shap_values)
#         mail_content = '{0}, {1}, {2} ok!! at se8'.format(shap_type,f,cj)
#         mensagem(mail_content)

    labels2 = labels[cj][msc_p,np.newaxis] == 0
    cf_fp, cf_size = labels2.sum(), labels2.shape[0]
    
    print('Usando somente a confiança: {:>6d} FP em {:>4d} exemplos ({:2.2f}%)'.format(cf_fp, cf_size, 100*cf_fp/cf_size))
    if modo == 'cos':
        if cj == 'train':
            VP = shap_values[labels[cj][msc_p] != 0].mean(axis=0)
            FP = shap_values[labels[cj][msc_p] == 0].mean(axis=0)
            vp_fp = np.array([VP, FP])

            # Similaridade entre a média dos gradientes (VP e FP) no conjunto treino
            print('Similaridade:',float(cosine_similarity(VP[np.newaxis,:], FP[np.newaxis,:])))
            # Normas das médias dos gradientes (VP e FP) no conjunto treino
            norma_vp_fp = np.linalg.norm(vp_fp, axis=1)
            print('||VP|| = {}\n||FP|| = {}'.format(norma_vp_fp[0],norma_vp_fp[1]))

        dados = np.concatenate((prediction[msc_p,np.newaxis],cosine_similarity(shap_values, vp_fp), np.linalg.norm(shap_values,axis=1, keepdims=True),labels2), axis=1)
        dados = pd.DataFrame(dados, index=features[cj][msc_p].index, columns=['Confiança','cos(\u03B8)\u2207_VP','cos(\u03B8)\u2207_FP','||\u2207||','FP?'])

        refin = 0					
        adv_fp, adv_size = (((dados['cos(\u03B8)\u2207_FP']-dados['cos(\u03B8)\u2207_VP'])>=refin) & dados['FP?']).sum(), ((dados['cos(\u03B8)\u2207_FP']-dados['cos(\u03B8)\u2207_VP'])>=refin).sum()
        # cf_idx = dl[cj][dl[cj]['FP?']].iloc[-adv_fp].name
        # cf_idx_size = cf_size - np.flatnonzero(dados.index == cf_idx).squeeze()
        print('Usando confiança + SHAP: {:>9d} FP em {:>4d} exemplos ({:2.2f}%)'.format(adv_fp, adv_size, 100*adv_fp/adv_size))
        # print('A partir do menos confiável: {:>5d} FP em {:>4d} exemplos ({:2.2f}%)\n'.format(adv_fp, cf_idx_size, 100*adv_fp/cf_idx_size), file=sumario)
        

    elif modo == 'pca':
        
        shap_values = features[cj][msc_p].values
        dados = np.concatenate((prediction[msc_p,np.newaxis],shap_values, labels2), axis=1)
        dados = pd.DataFrame(dados, index=features[cj][msc_p].index, columns=['Confiança']+features[cj].columns.tolist()+['FP?'])

    elif modo == 'pca_9':
        if cj == 'train':
            from sklearn.decomposition import PCA
            n=9
            pca = PCA(n_components=n, random_state=seed)
            pca.fit(shap_values)

        shap_values = pca.transform(shap_values)
        dados = np.concatenate((prediction[msc_p,np.newaxis],shap_values, labels2), axis=1)
        dados = pd.DataFrame(dados, index=features[cj][msc_p].index, columns=['Confiança']+['pca'+str(i) for i in range(n)]+['FP?'])

    dados.sort_values('Confiança',ascending=False,inplace=True)
    dados = dados.astype({'FP?': 'bool'})

    dl[cj] = dados



labels['train'] = dl['train']['FP?']
features['train'] = dl['train'].iloc[:, :-1]
# LE.fit(labels['train'])
# labels['train'] = LE.transform(labels['train'])
scaler.fit(features['train'])
features['train'] = scaler.transform(features['train'])
labels['val'] = dl['val']['FP?']
features['val'] = dl['val'].iloc[:, 0:-1]
# labels['val'] = LE.transform(labels['val'])
features['val'] = scaler.transform(features['val'])

labels['test'] = dl['test']['FP?']
features['test'] = dl['test'].iloc[:, 0:-1]
# labels['test'] = LE.transform(labels['test'])
features['test'] = scaler.transform(features['test'])

cj='test'
if (dataset == 'TrafficLabelling/') & (f in ['C','CPd_dest']):
    p = np.linspace(0,.25,11)
else:
    p = np.linspace(0,1,11)
fracoes = dl[cj].shape[0]*p
fracoes = fracoes.round().astype(int)[1:]
# print(p)
# print(fracoes)

tot_fp = msc_fp.sum()
dx = p[2]-p[1]
cj_analise = np.zeros(fracoes.shape,dtype='int')
for i,j in enumerate(fracoes):
    cj_analise[i] = dl[cj].iloc[-j:,-1].sum()
# print(cj_analise)


print('\nAUC: {:.3f} (sem 2ª rede)'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)))



rodadas = 20

if modo == 'pca_9':
    atbs = np.floor(np.linspace(0,dados.shape[1]-1,4)[1:]).astype(int)
else:
    atbs = [dados.shape[1]-1]# colocar 1 antes

# atb = dados.shape[1]-1 # 1 para apenas confinça, 4 para confiança + aprox adv
# atb = 1 # 1 para apenas confinça, 4 para confiança + aprox adv
for atb in atbs[-1:]:
    cj_analise = np.zeros(fracoes.shape)

    if segunda == 'mlp':
        dl2 = np.zeros(labels[cj].shape)
        for n in range(rodadas):
            model2 = tf.keras.models.Sequential([tf.keras.layers.Dense(10, input_dim=features['train'][:,:atb].shape[1], activation='relu'), tf.keras.layers.Dense(10, activation='relu'), tf.keras.layers.Dense(1, activation='sigmoid')])

            model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)
            model2.fit(features['train'][:,:atb], labels['train'], epochs=150, verbose=0, batch_size = 256, validation_data=(features['val'][:,:atb], labels['val']), callbacks=[es])

            dl2 += model2.predict(features[cj][:,:atb]).squeeze()

        dl2 = dl2/rodadas
        msg = ''

    elif segunda == 'knn':
        from sklearn.neighbors import KNeighborsClassifier
        fracoes_val = dl['val'].shape[0]*p
        fracoes_val = fracoes_val.round().astype(int)[1:]
        # cj_analise = np.zeros(fracoes.shape)
        aucs = np.zeros(rodadas)
        for l in range(1, rodadas):
            model2 = KNeighborsClassifier(n_neighbors=l)
            model2.fit(features['train'][:,1:atb], labels['train'])
            dl2 = model2.predict_proba(features['val'][:,1:atb])
            dl2 = pd.DataFrame(np.column_stack((dl2[:,1],labels['val'])),index=labels['val'].index,columns=['Conf2',labels['val'].name])
            dl2.sort_values(by=['Conf2'], inplace=True)
            for i,j in enumerate(fracoes_val):
                cj_analise[i] = dl2.iloc[-j:,-1].sum()
            aucs[l] = np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)

        n_neighbors=aucs.argmax()
        model2 = KNeighborsClassifier(n_neighbors=n_neighbors)
        model2.fit(features['train'][:,1:atb], labels['train'])
        dl2 = model2.predict_proba(features['test'][:,1:atb])[:,1]
        msg = '\nneighbors={:d}'.format(n_neighbors)

    dl2 = pd.DataFrame(np.column_stack((dl2,labels[cj])),index=labels[cj].index,columns=['Conf2',labels[cj].name])
    dl2.sort_values(by=['Conf2'], inplace=True)
    for i,j in enumerate(fracoes):
        cj_analise[i] = dl2.iloc[-j:,-1].sum()
    
    print('{:s}\nFP total no dl: {:d} de {:d}\n'.format(msg,cf_fp,tot_fp))
    print(np.column_stack((cj_analise, cj_analise/tot_fp)))
    print('\nAUC: {:.3f} (atb = {:d})'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx), atb))
    print('\n------------------------------------------------------------\n')
    

Usando somente a confiança:     26 FP em 1152 exemplos (2.26%)
Usando somente a confiança:      7 FP em  290 exemplos (2.41%)
Usando somente a confiança:      9 FP em  353 exemplos (2.55%)

AUC: 0.753 (sem 2ª rede)

neighbors=11
FP total no dl: 9 de 9

[[8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [9.         1.        ]
 [9.         1.        ]
 [9.         1.        ]
 [9.         1.        ]
 [9.         1.        ]
 [9.         1.        ]]

AUC: 0.957 (atb = 90)

------------------------------------------------------------



In [3]:
# dl2 = dl2_final
# dl2 = dl2_adv
dl2.columns = ['Conf2','FP?']
tot_confs = dl2['Conf2'].value_counts()
tot_confs.sort_index(ascending=False,inplace=True)
fp_confs = dl2[dl2['FP?'] == 1]['Conf2'].value_counts()
tot_conf_counter = 1
for i,j in enumerate(fracoes):
  while j > tot_confs.iloc[:tot_conf_counter].sum():
    tot_conf_counter += 1
  fp_ant = fp_confs[fp_confs.index > tot_confs.index[tot_conf_counter-1]].sum()
  residuo = j - tot_confs.iloc[:tot_conf_counter-1].sum()
  if np.isin(tot_confs.index[tot_conf_counter-1],fp_confs.index,assume_unique=True):
    dens = fp_confs[tot_confs.index[tot_conf_counter-1]]/tot_confs.iloc[tot_conf_counter-1]
  else:
    dens=0
  # dens = fp_confs[tot_confs.index[tot_conf_counter-1]]/tot_confs.iloc[tot_conf_counter-1]
  cj_analise[i] = fp_ant + residuo*dens
cj_analise = np.floor(cj_analise)
print('\natb: {:d}\nFP total no dl: {:d} de {:d}\n'.format(atb,cf_fp,tot_fp))
print('AUC: {:.3f}\n'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)))
print(np.column_stack((cj_analise, cj_analise/tot_fp)))


atb: 90
FP total no dl: 9 de 9

AUC: 0.883

[[6.         0.66666667]
 [8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [8.         0.88888889]
 [9.         1.        ]]


In [ ]:
dl2_path = 'C:/Users/SE8ALUNO/maj_ricardo/google/doutorado/duvidas/'+dataset+'shap/dl2_'+f+'.csv'

# dl2.rename(columns={'Conf2':'Conf2_shap','FP?':'FP?_shap'}, inplace=True)
# dl2.to_csv(dl2_path)
dl2 = pd.read_csv(dl2_path,index_col=0,float_precision='round_trip')

for i,j in enumerate(fracoes):
  cj_analise[i] = dl2.iloc[-j:,-1].sum()
print('AUC: {:.3f}\n'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)))

In [ ]:
# dl2.rename(columns={'Conf2':'Conf2_shap','FP?':'FP?_shap'}, inplace=True)

In [ ]:
dl2_path_adv = 'C:/Users/SE8ALUNO/maj_ricardo/google/doutorado/duvidas/'+dataset+'adv/dl2_'+f+'.csv'

dl2_adv = pd.read_csv(dl2_path_adv,index_col=0,float_precision='round_trip')

for i,j in enumerate(fracoes):
  cj_analise[i] = dl2_adv.iloc[-j:,-1].sum()
# print('\natb: {:d}\nFP total no dl: {:d} de {:d}\n'.format(atb,cf_fp,tot_fp))
print('AUC: {:.3f}\n'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)))
# print(np.column_stack((cj_analise, cj_analise/tot_fp)))

In [ ]:
dl2_final = pd.concat([dl2, dl2_adv], axis=1)
assert dl2.shape[0] == dl2_adv.shape[0], 'dls devem ter o mesmo tamanho'
assert dl2_final.shape[0] == dl2.shape[0], 'indexes devem ser os mesmos'

In [ ]:
dl2_final = pd.concat([dl2, dl2_adv], axis=1)
dl2_final = pd.concat([dl2_final[['Conf2_shap','Conf2_adv']].mean(axis=1),dl2_final['FP?_adv']], axis=1)
dl2_final.columns = ['Conf2','FP?']
dl2_final.sort_values(by=['Conf2'], inplace=True)
for i,j in enumerate(fracoes):
  cj_analise[i] = dl2_final.iloc[-j:,-1].sum()
# print('\natb: {:d}\nFP total no dl: {:d} de {:d}\n'.format(atb,cf_fp,tot_fp))
print('AUC: {:.3f}\n'.format(np.trapz(cj_analise/tot_fp, dx=dx)/((len(cj_analise)-1)*dx)))
print(np.column_stack((cj_analise, cj_analise/tot_fp)))